In [1]:
from QuantLib import *
import QuantLib as ql
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import sys, os
cwd = os.getcwd()
sys.path.append(cwd)

In [2]:
from modules.cashflow import Cashflow

ImportError: cannot import name 'Cashflow'

TypeError: __init__() missing 1 required positional argument: 'cashflow'

In [106]:
ay21 = pd.read_excel(open('../Portfolio.xlsx', 'rb'), sheet_name='AY21')[['Fecha de Pago', 'Cash Flow']]
ay24 = pd.read_excel(open('../Portfolio.xlsx', 'rb'), sheet_name='AY24')[['Fecha de Pago', 'Cash Flow']]
aa25d = pd.read_excel(open('../Portfolio.xlsx', 'rb'), sheet_name='AA25D')[['Fecha de Pago', 'Cash Flow']]

In [107]:
sheet_names = ['AY21', 'AY24', 'TO21', 'AA25D']
dfs = []
for sheet_name in sheet_names:
    new_df = pd.read_excel(open('../Portfolio.xlsx', 'rb'), sheet_name=sheet_name)[['Fecha de Pago', 'Cash Flow']]
    new_df['Bond'] = sheet_name
    new_df['Cash Flow'] = new_df['Cash Flow'].apply(lambda cf: cf.replace(',','.'))
    new_df['Cash Flow'] = pd.to_numeric(new_df['Cash Flow'])
    new_df['Price'] = -new_df.iloc[0]['Cash Flow']
    new_df = new_df.drop(0)
    new_df['Fecha de Pago'] = pd.to_datetime(new_df['Fecha de Pago'])
    dfs.append(new_df)
df = pd.concat(dfs, 0)

In [108]:
today = datetime.datetime.today()

todaysDate = ql.Date(today.weekday(), today.month, today.year)
ql.Settings.instance().evaluationDate = todaysDate

### TO21

In [125]:
# Pricing of bond
to21 = df[df.Bond=='TO21']

calendar = ql.Argentina()
dayCount = ql.Thirty360()
compounding = ql.Compounded

compounding = ql.Compounded
compoundingFrequency = ql.Semiannual
tenor = ql.Period(compoundingFrequency)
faceValue = 100
settlementDays = 0

coupons = [x/100 for x in to21['Cash Flow']]
b = ql.FixedRateBond(settlementDays, calendar, faceValue, 
                     ql.Date(1, 9, 2019), ql.Date(3, 10, 2021), tenor, coupons, dayCount, ql.Unadjusted, ql.Unadjusted)

market_price = to21['Price'].values[0]

# DEBERIA DAR: 45.93%
to21.T
print ('PAYMENTS:', [cf.date() for cf in b.cashflows()])
print ('COUPONS:', [round(cf.amount(),2) for cf in b.cashflows()])
ytm = b.bondYield(market_price, dayCount, compounding, compoundingFrequency)
print ('YTM:', ytm)


,1,2,3,4,5
Fecha de Pago,2019-03-10 00:00:00,2020-03-04 00:00:00,2020-03-10 00:00:00,2021-03-04 00:00:00,2021-03-10 00:00:00
Cash Flow,9.1,9.1,9.1,9.1,109.1
Bond,TO21,TO21,TO21,TO21,TO21
Price,72,72,72,72,72


PAYMENTS: [Date(3,10,2019), Date(3,4,2020), Date(3,10,2020), Date(3,4,2021), Date(3,10,2021), Date(3,10,2021)]
COUPONS: [0.81, 4.55, 4.55, 4.55, 54.55, 100.0]
YTM: 0.4418171187662839


In [119]:
# b.cashflows()[4].amount()

54.55000000000001

In [111]:
# schedule = ql.Schedule(ql.Date(1, 9, 2019), ql.Date(3, 10, 2021), tenor, calendar, 
#                        ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Backward, False)
# print(list(schedule))

In [93]:
# ql.FixedRateBondHelper(ql.SimpleQuote(faceValue), 
#                        settlementDays, 
#                        faceValue, 
#                        schedule, 
#                        coupons, dayCount)

In [130]:
# FixedRateBondPtr::FixedRateBondPtr(
#     Integer,Calendar const &,Real,Date const &,Date const &,Period const &,
#     std::vector< Rate,std::allocator< Rate > > const &,DayCounter const &,BusinessDayConvention,BusinessDayConvention,
#     Real, Date const &,Date const &,DateGeneration::Rule,bool,Calendar const &,Period const &,Calendar const &,
#     BusinessDayConvention const,bool)

# FixedRateBond (Natural settlementDays, Real faceAmount, const Schedule &schedule, const std::vector< InterestRate > &coupons, 
#                BusinessDayConvention paymentConvention=Following, Real redemption=100.0, const Date &issueDate=Date(), 
#                const Calendar &paymentCalendar=Calendar(), const Period &exCouponPeriod=Period(), 
#                const Calendar &exCouponCalendar=Calendar(), const BusinessDayConvention exCouponConvention=Unadjusted, 
#                bool exCouponEndOfMonth=false)

# FixedRateBond (Natural settlementDays, const Calendar &couponCalendar, Real faceAmount, const Date &startDate, const Date &maturityDate, 
#                const Period &tenor, const std::vector< Rate > &coupons, const DayCounter &accrualDayCounter, 
#                BusinessDayConvention accrualConvention=Following, BusinessDayConvention paymentConvention=Following, Real redemption=100.0, 
#                const Date &issueDate=Date(), const Date &stubDate=Date(), DateGeneration::Rule rule=DateGeneration::Backward, 
#                bool endOfMonth=false, const Calendar &paymentCalendar=Calendar(), const Period &exCouponPeriod=Period(), 
#                const Calendar &exCouponCalendar=Calendar(), const BusinessDayConvention exCouponConvention=Unadjusted, 
#                bool exCouponEndOfMonth=false)

accrual_convention = ql.Unadjusted
payment_convention = ql.Unadjusted
b = ql.FixedRateBond(settlementDays, calendar, faceValue, ql.Date(1, 9, 2019), ql.Date(3, 10, 2021), 
                     tenor, coupons, dayCount, 
                     accrual_convention, payment_convention, 100,
                    )#,
                     #faceValue, ql.Date(1, 9, 2019), ql.Date(3, 10, 2021), False, calendar, tenor, calendar, ql.Unadjusted, False)

print ('PAYMENTS:', [cf.date() for cf in b.cashflows()])
print ('COUPONS:', [round(cf.amount(),2) for cf in b.cashflows()])

PAYMENTS: [Date(3,10,2019), Date(3,4,2020), Date(3,10,2020), Date(3,4,2021), Date(3,10,2021), Date(3,10,2021)]
COUPONS: [0.81, 4.55, 4.55, 4.55, 54.55, 100.0]


In [188]:
Settings.instance().evaluationDate = Date(3,7,2019)

schedule = Schedule(Date(3,4,2019), Date(3,10,2021), Period(6,Months),
                    NullCalendar(), Following, Following,
                    DateGeneration.Forward, False)

# day_convention = Thirty360(ActualActual.ISMA)
day_convention = Thirty360(Thirty360.BondBasis)

coupons = [2*x/100 for x in to21['Cash Flow']]
coupons[4] = 2*9.1/100

bond = FixedRateBond(0, 100.0, schedule, coupons, day_convention)

print(bond.bondYield(72, day_convention, Simple, Semiannual))
print(bond.cleanPrice(0.46, day_convention, Simple, Semiannual))

print ('PAYMENTS:', [cf.date() for cf in bond.cashflows()])
print ('COUPONS:', [round(cf.amount(),2) for cf in bond.cashflows()])

0.373337044395566
62.77647757816803
PAYMENTS: [Date(3,10,2019), Date(3,4,2020), Date(3,10,2020), Date(3,4,2021), Date(3,10,2021), Date(3,10,2021)]
COUPONS: [9.1, 9.1, 9.1, 9.1, 9.1, 100.0]


In [186]:
coupons

[0.182, 0.182, 0.182, 0.182, 2.182]

In [168]:
Settings.instance().evaluationDate = Date(6,3,2008)

schedule = Schedule(Date(6,3,2008), Date(6,3,2010), Period(6,Months),
                    NullCalendar(), Following, Following,
                    DateGeneration.Forward, False)

bond = FixedRateBond(0, 100.0, schedule, [0.02], ActualActual(ActualActual.ISMA))

print(bond.cleanPrice(0.02, ActualActual(ActualActual.ISMA), Simple, Semiannual))
print ('PAYMENTS:', [cf.date() for cf in bond.cashflows()])
print ('COUPONS:', [round(cf.amount(),2) for cf in bond.cashflows()])

100.0
PAYMENTS: [Date(6,9,2008), Date(6,3,2009), Date(6,9,2009), Date(6,3,2010), Date(6,3,2010)]
COUPONS: [1.0, 1.0, 1.0, 1.0, 100.0]


### AA25D (ver caso de schedule con intervalos no todos iguales)

In [75]:
# Pricing of bond
aa25d = df[df.Bond=='AA25D']

calendar = ql.Argentina()
dayCount = ql.Thirty360()
compounding = ql.Compounded

compounding = ql.Compounded
compoundingFrequency = ql.Semiannual
tenor = ql.Period(compoundingFrequency)
faceValue = 100
settlementDays = 0

coupons = [x/100 for x in aa25d['Cash Flow']]
schedule = ql.Schedule(ql.Date(18, 10, 2019), ql.Date(18, 4, 2025), tenor, calendar, 
                       ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Backward, False)

b = ql.FixedRateBond(settlementDays, calendar, faceValue, ql.Date(1, 9, 2019), ql.Date(3, 10, 2025), tenor, coupons, 
                     dayCount, ql.Unadjusted, ql.Unadjusted)

market_price = aa25d['Price'].values[0]

# DEBERIA DAR: 14.95%
aa25d.T
print ('PAYMENTS:', [cf.date() for cf in b.cashflows()])
print ('COUPONS:', [round(cf.amount(),2) for cf in b.cashflows()])
print ('Market Price:', market_price)

ytm = b.bondYield(market_price, dayCount, compounding, compoundingFrequency)
print ('YTM:', ytm)

,1,2,3,4,5,6,7,8,9,10,11,12
Fecha de Pago,2019-10-18 00:00:00,2020-04-18 00:00:00,2020-10-18 00:00:00,2021-04-18 00:00:00,2021-10-18 00:00:00,2022-04-18 00:00:00,2022-10-18 00:00:00,2023-04-18 00:00:00,2023-10-18 00:00:00,2024-04-18 00:00:00,2024-10-18 00:00:00,2025-04-18 00:00:00
Cash Flow,2.88,2.88,2.88,2.88,2.88,2.88,2.88,35.88,1.93,34.93,0.98,34.98
Bond,AA25D,AA25D,AA25D,AA25D,AA25D,AA25D,AA25D,AA25D,AA25D,AA25D,AA25D,AA25D
Price,73.22,73.22,73.22,73.22,73.22,73.22,73.22,73.22,73.22,73.22,73.22,73.22


PAYMENTS: [Date(3,10,2019), Date(3,4,2020), Date(3,10,2020), Date(3,4,2021), Date(3,10,2021), Date(3,4,2022), Date(3,10,2022), Date(3,4,2023), Date(3,10,2023), Date(3,4,2024), Date(3,10,2024), Date(3,4,2025), Date(3,10,2025), Date(3,10,2025)]
COUPONS: [0.26, 1.44, 1.44, 1.44, 1.44, 1.44, 1.44, 17.94, 0.96, 17.46, 0.49, 17.49, 17.49, 100.0]
Market Price: 73.22
YTM: 0.17270562211137716
